In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPool1D, Flatten, Dropout,LSTM

In [ ]:
# estimator = [('RF', RandomForestClassifier(n_estimators = 200, max_depth = 5)), ('GB', GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 50)),
#              ('CAT', CatBoostClassifier(depth= 5, iterations = 35, learning_rate = 0.35)), ('ADB', AdaBoostClassifier(n_estimators = 200, learning_rate = 0.1, random_state = 50))]
# Stacking = StackingClassifier( estimators=estimator, final_estimator= CatBoostClassifier(depth= 5, iterations = 35, learning_rate = 0.35))

**Train**

In [ ]:
df = pd.read_csv('/content/FastText+PAAC.csv')
columns = df.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
X = df[columns]
Y = df[target]

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.3)

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 200, max_depth = 5),
          XGBClassifier(n_estimators = 200,max_depth = 5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 5,random_state = 50),
          GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 50),
          AdaBoostClassifier(n_estimators = 200, learning_rate = 0.1, random_state = 50)]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics((FT+PAAC)-CV).csv")
# clf = StackingClassifier( estimators=estimator, final_estimator=RandomForestClassifier(n_estimators = 200, max_depth = 5))
# prob = clf.fit_transform(xtrain, ytrain)
# pd.DataFrame(prob).to_csv("total_Metics_Probability.csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=5, n_estimato...  0.856209  0.716010   
1  XGBClassifier(base_score=None, booster=None, c...  0.869281  0.738234   
2       LGBMClassifier(max_depth=5, random_state=50)  0.852941  0.706084   
3  GradientBoostingClassifier(learning_rate=0.5, ...  0.866013  0.732257   
4  AdaBoostClassifier(learning_rate=0.1, n_estima...  0.843137  0.688024   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.713019   0.818750  0.897260  0.856209      0.81875     0.897260  
1  0.738171   0.858108  0.869863  0.863946      0.86875     0.869863  
2  0.705706   0.834437  0.863014  0.848485      0.84375     0.863014  
3  0.731866   0.847682  0.876712  0.861953      0.85625     0.876712  
4  0.686556   0.814103  0.869863  0.841060      0.81875     0.869863  


**Test**

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

# cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 200, max_depth = 5),
          XGBClassifier(n_estimators = 200,max_depth = 5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 5,random_state = 50),
          GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 50),
          AdaBoostClassifier(n_estimators = 200, learning_rate = 0.1, random_state = 50)]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  model.fit(xtrain, ytrain)
  pred = model.predict(xtest)
  # pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytest, pred)
  mcc = matthews_corrcoef(ytest, pred)
  cm1 = confusion_matrix(ytest, pred)
  kappa = cohen_kappa_score(ytest, pred)
  f1 = f1_score(ytest, pred)
  precision_score = precision_score(ytest, pred)
  recall_score = recall_score(ytest, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics((FT+PAAC)-TS)).csv")
# clf = StackingClassifier( estimators=estimator, final_estimator=RandomForestClassifier(n_estimators = 200, max_depth = 5))
# prob = clf.fit_transform(xtest, ytest)
# pd.DataFrame(prob).to_csv("total_Metics_Probability(LSA+PAAC).csv")

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 146, number of negative: 160
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001323 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14426
[LightGBM] [Info] Number of data points in the train set: 306, number of used features: 150
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.477124 -> initscore=-0.091567
[LightGBM] [Info] Start training from score -0.091567
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

# **CNN**

In [ ]:
df = pd.read_csv('/content/Res-FastText+PAAC.csv')
columns = df.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
X = df[columns]
Y = df[target]

**Train**

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.3)

In [ ]:
xtrain = xtrain.to_numpy()
ytrain = ytrain.to_numpy()
xtrain = xtrain.reshape(xtrain.shape[0], xtrain.shape[1], 1)
# xtest = xtest.reshape(xtest.shape[0], xtest.shape[1], 1)

In [ ]:
kf = KFold(n_splits=5, shuffle=True)
for train_index, val_index in kf.split(xtrain):
    X_train, X_val = xtrain[train_index], xtrain[val_index]
    y_train, y_val = ytrain[train_index], ytrain[val_index]

In [ ]:
cnn = Sequential()

In [ ]:
cnn.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)))
cnn.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
cnn.add(Conv1D(filters=128, kernel_size=2, activation='relu'))

In [ ]:
cnn.add(MaxPool1D(pool_size=2))

In [ ]:
cnn.add(Flatten())

In [ ]:
cnn.add(Dense(128, activation='relu'))
cnn.add(Dense(64, activation='relu'))
cnn.add(Dense(1, activation='sigmoid'))

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
cnn.fit(X_train, y_train, epochs = 40, batch_size= 64)

Epoch 1/40
4/4 [==============================] - 2s 90ms/step - loss: 0.6507 - accuracy: 0.4735
Epoch 2/40
4/4 [==============================] - 0s 92ms/step - loss: 0.5500 - accuracy: 0.7020
Epoch 3/40
4/4 [==============================] - 0s 95ms/step - loss: 0.4292 - accuracy: 0.8531
Epoch 4/40
4/4 [==============================] - 0s 94ms/step - loss: 0.3579 - accuracy: 0.8571
Epoch 5/40
4/4 [==============================] - 1s 243ms/step - loss: 0.3305 - accuracy: 0.8939
Epoch 6/40
4/4 [==============================] - 1s 101ms/step - loss: 0.2836 - accuracy: 0.8857
Epoch 7/40
4/4 [==============================] - 1s 231ms/step - loss: 0.2601 - accuracy: 0.9102
Epoch 8/40
4/4 [==============================] - 1s 100ms/step - loss: 0.2410 - accuracy: 0.9143
Epoch 9/40
4/4 [==============================] - 0s 98ms/step - loss: 0.2296 - accuracy: 0.9184
Epoch 10/40
4/4 [==============================] - 0s 104ms/step - loss: 0.2115 - accuracy: 0.9265
Epoch 11/40
4/4 [=======

In [ ]:
pred = cnn.predict(X_val)
y_pred_classes = np.round(pred).astype(int)

2/2 [==============================] - 0s 30ms/step


In [ ]:
accuracy_score(y_val, y_pred_classes), f1_score(y_val, y_pred_classes), cohen_kappa_score(y_val, y_pred_classes), matthews_corrcoef(y_val, y_pred_classes)

(0.9344262295081968,
 0.9310344827586207,
 0.8685344827586207,
 0.8685344827586207)

In [ ]:
cm1 = confusion_matrix(y_val, y_pred_classes)
specificity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
sensitivity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
specificity, sensitivity

(0.9375, 0.9310344827586207)

**Test**

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.3)

In [ ]:
sample_size = xtrain.shape[0] # number of samples in train set
time_steps  = xtrain.shape[1] # number of features in train set
input_dimension = 1               # each feature is represented by 1 number

In [ ]:
train_data_reshaped = xtrain.values.reshape(sample_size,time_steps,input_dimension)
n_timesteps = train_data_reshaped.shape[1]
n_features  = train_data_reshaped.shape[2]

In [ ]:
cnn = Sequential()
cnn.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)))
cnn.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
cnn.add(Conv1D(filters=128, kernel_size=2, activation='relu'))

In [ ]:
cnn.add(MaxPool1D(pool_size=4))

In [ ]:
cnn.add(Flatten())

In [ ]:
cnn.add(Dense(128, activation='relu'))
cnn.add(Dense(64, activation='relu'))
cnn.add(Dense(1, activation='sigmoid'))

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
cnn.fit(xtrain, ytrain, epochs = 40, batch_size= 64)

Epoch 1/40
5/5 [==============================] - 2s 86ms/step - loss: 0.6560 - accuracy: 0.4902
Epoch 2/40
5/5 [==============================] - 0s 85ms/step - loss: 0.5168 - accuracy: 0.8268
Epoch 3/40
5/5 [==============================] - 0s 81ms/step - loss: 0.4143 - accuracy: 0.8235
Epoch 4/40
5/5 [==============================] - 0s 80ms/step - loss: 0.3283 - accuracy: 0.8889
Epoch 5/40
5/5 [==============================] - 0s 80ms/step - loss: 0.3190 - accuracy: 0.8954
Epoch 6/40
5/5 [==============================] - 0s 85ms/step - loss: 0.3017 - accuracy: 0.8987
Epoch 7/40
5/5 [==============================] - 0s 80ms/step - loss: 0.2889 - accuracy: 0.8856
Epoch 8/40
5/5 [==============================] - 0s 85ms/step - loss: 0.3070 - accuracy: 0.8889
Epoch 9/40
5/5 [==============================] - 0s 90ms/step - loss: 0.2664 - accuracy: 0.8987
Epoch 10/40
5/5 [==============================] - 0s 88ms/step - loss: 0.2537 - accuracy: 0.9150
Epoch 11/40
5/5 [============

In [ ]:
pred = cnn.predict(xtest)
pred = (pred > 0.5)

5/5 [==============================] - 0s 12ms/step


In [ ]:
accuracy_score(ytest, pred), f1_score(ytest, pred), cohen_kappa_score(ytest, pred), matthews_corrcoef(ytest, pred)

(0.9696969696969697,
 0.9714285714285714,
 0.9391845196959225,
 0.9396190204353759)

In [ ]:
cm1 = confusion_matrix(ytest, pred)
specificity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
sensitivity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
specificity, sensitivity

(0.9523809523809523, 0.9855072463768116)